In [2]:
import json
import pandas as pd
import os
import music21 as m21
import numpy as np

In [3]:
list_of_confs_v101_noisy = []
with open('../MTRCV101/MTRCV101NOISY/seen_confs_GridSearch_NOISY_MTRC_v101.json') as json_file:
    data = json.load(json_file)
    for conf in data:
        list_of_confs_v101_noisy.append((data[conf], conf))
list_of_confs_v101_noisy.sort()
list_of_confs_v101_noisy[:30]

[(0.19256838164030637, '11,7,81,1'),
 (0.21190467013049083, '21,8,70,1'),
 (0.22448002236265147, '1,23,73,3'),
 (0.22883257776056434, '8,19,66,7'),
 (0.23158762797273322, '21,14,62,3'),
 (0.23300709763274288, '30,5,62,3'),
 (0.2335346470664991, '16,20,61,3'),
 (0.2439805289474478, '24,13,60,3'),
 (0.24590556982917428, '43,1,52,4'),
 (0.2516084213651179, '37,6,49,8'),
 (0.2521097900456656, '29,15,53,3'),
 (0.2598821063942687, '3,14,62,21'),
 (0.26099979979463106, '7,3,64,26'),
 (0.26276805478563675, '32,12,45,11'),
 (0.26313418854964465, '39,7,40,14'),
 (0.2662986114946413, '19,23,52,6'),
 (0.2699244359893652, '22,2,51,25'),
 (0.2716330967472435, '20,11,48,21'),
 (0.27184057136415707, '17,3,54,26'),
 (0.27233569658054424, '9,38,52,1'),
 (0.27612212279174647, '36,1,39,24'),
 (0.2763296347932077, '17,3,53,27'),
 (0.27984366294066715, '41,15,42,2'),
 (0.27984679817603303, '58,1,35,6'),
 (0.2810924868092981, '30,18,45,7'),
 (0.28257808370356974, '17,4,49,30'),
 (0.2846080063843052, '47,11,3

In [4]:
def get_weigths_from_encoded_conf(encoded_conf):
    return tuple(map(lambda x: x / 100, map(float, encoded_conf.split(","))))

In [5]:
(W1, W2, W3, W4) = get_weigths_from_encoded_conf(list_of_confs_v101_noisy[0][1])
(W1, W2, W3, W4)

(0.11, 0.07, 0.81, 0.01)

In [6]:
# Function to retrieve a list of midi pitch events and its timestamp
def getMelodyDeltaTimes(eventsintrack):
    
    # Initialize array
    DeltaTimes = []
    
    # Initialize cumulative sum
    cum_sum = 0
    
    # Initialize variable to track the time delta
    prev_deltatime = 0
    
    # Traverse the events
    for ev in eventsintrack:
        
        # If a note starts
        if (ev.isNoteOn()):
            
            # Get the pitch name and save it with the cumulative sum, midi pitch and name
            pitch_in_time = m21.pitch.Pitch(ev.pitch)
            DeltaTimes.append((cum_sum, prev_deltatime, pitch_in_time.midi, pitch_in_time.spanish, pitch_in_time))
            
            # Restart the delta time
            prev_deltatime = 0
        
        # Else if there is a delta time
        elif(str(ev.type) == "DeltaTime"):
            
            # We sum the time
            cum_sum += ev.time
            
            # We sum it to the current delta time
            prev_deltatime += ev.time
    
    # Return the array
    return DeltaTimes

In [7]:
def getSongKey(song):
    key = song.analyze("key")
    return key

In [8]:
def getSongKeyFromMelody_W_Times(melody_w_times_in_k):
    sc_test = m21.stream.Score()
    p0_test = m21.stream.Part()
    p0_test.id = 'part0'
    for pitch_i in melody_w_times_in_k:
        n_i = m21.note.Note(pitch_i[4])
        p0_test.append(n_i)
    sc_test.insert(0, p0_test)
    return getSongKey(sc_test)

In [9]:
def get_SCLM_v100(melody_w_times_A, melody_w_times_B):
    
    # We use a Dynamic Programming approach
    max_len = max(len(melody_w_times_A), len(melody_w_times_B)) + 1
    
    # memoization array
    memo = np.full(shape=(max_len,max_len), fill_value=-1)
    
    # Get the limits for each melody
    lim_A = len(melody_w_times_A)
    lim_B = len(melody_w_times_B)
    
    # Actual DP implementation
    for i in range(lim_A, -1, -1):
        for j in range(lim_B, -1, -1):
            
            # If we are at the limits the solution is 0
            if i == lim_A or  j == lim_B:
                memo[i][j] = 0
                continue
            
            # If there is a match a possible solution is the previous plus one
            curr_value = 0
            if melody_w_times_A[i][3] == melody_w_times_B[j][3]:
                curr_value = memo[i + 1][j + 1] + 1
            
            # The actual solution is the maximum between the one if there is a match, or skip on the melody A or melody B
            curr_value = max(curr_value, max(memo[i + 1][j], memo[i][j + 1]))
            
            # Save the solution
            memo[i][j] = curr_value
    
    # With the memoization table we can retrieve the actual melody
    i = 0
    j = 0
    SCLM = []
    while i != lim_A and j != lim_B:
    
        if ((memo[i + 1][j + 1] + 1) == memo[i][j]):
            SCLM.append((i, j))
            i += 1
            j += 1
        elif (memo[i + 1][j] == memo[i][j]):
            i += 1
        elif (memo[i][j + 1] == memo[i][j]):
            j += 1
    
    return SCLM

In [10]:
def get_max_timestamp_dif(melody_w_times_A, melody_w_times_B):
    return max(
        melody_w_times_A[len(melody_w_times_A) - 1][0] - melody_w_times_A[0][0],
        melody_w_times_B[len(melody_w_times_B) - 1][0] - melody_w_times_B[0][0]
    )

In [11]:
def getDifSCLM(melody_w_times_A, melody_w_times_B, sclm):
    
    # If there is no sclm or it is just one return max possible value
    if (len(sclm) <= 1):
        return get_max_timestamp_dif(melody_w_times_A, melody_w_times_B)
    
    
    # Initialize the arrays
    T_A = np.zeros(shape=(len(sclm) - 1))
    T_B = np.zeros(shape=(len(sclm) - 1))
    T_C = np.zeros(shape=(len(sclm) - 1))
    Dif_ = np.zeros(shape=(len(sclm) - 1))
    
    for i in range(1, len(sclm)):
        T_A[i - 1] = melody_w_times_A[sclm[i][0]][0] - melody_w_times_A[sclm[i-1][0]][0]
        T_B[i - 1] = melody_w_times_B[sclm[i][1]][0] - melody_w_times_B[sclm[i-1][1]][0]
        T_C[i - 1] = np.abs(T_A[i - 1] - T_B[i - 1])
    
    T_C_mean = np.mean(T_C)
    
    for i in range(0, len(T_B)):
        T_B[i] += T_C_mean
        Dif_[i] = T_A[i] - T_B[i]
    
    return T_C_mean
    

In [12]:
def get_MTRC_v100_from_melody_w_times(melody_w_times_A, melody_w_times_B):
    
    # Assert at least one element for each melody
    if (len(melody_w_times_A) == 0 or len(melody_w_times_B) == 0):
        return 1
    
    # Initialize result variable
    result_value = 0
    
    # Get Keys
    key_A = getSongKeyFromMelody_W_Times(melody_w_times_A)
    key_B = getSongKeyFromMelody_W_Times(melody_w_times_B)
    
    # D1: Scale  
    scale_dif1 = 0
    if (key_A.name != key_B.name):
        scale_dif1 = W1
    result_value += scale_dif1
    
    # D2: Mode  
    mode_dif2 = 0
    if (key_A.mode != key_B.mode):
        mode_dif2 = W2
    result_value += mode_dif2
    
    # Get SCLM v100
    sclm = get_SCLM_v100(melody_w_times_A, melody_w_times_B)
    
    # Get max len
    max_len = max(len(melody_w_times_A), len(melody_w_times_B))
    
    # D3: SCLM Length
    sclmlen_dif3 = ((max_len - len(sclm)) / max_len) * W3
    result_value += sclmlen_dif3
    
    # Get the Diff on temporal spacing in the SCLM
    dif_sclm = getDifSCLM(melody_w_times_A, melody_w_times_B, sclm)
    
    # D4: dif in sclm
    max_timestamp_dif = get_max_timestamp_dif(melody_w_times_A, melody_w_times_B)
    sclmdif_dif4 = (dif_sclm / max_timestamp_dif) * W4
    result_value += sclmdif_dif4
    
    return result_value

In [13]:
# Define dataset path
# DATASET_PATH = "/home/sirivasv/Documents/MuseScore3/Scores/MTRCV100/MTC-ANN-2.0.1/mid/"
DATASET_PATH = "/media/sirivasv/JASON/Saul/MCC/DATASETS/DATASUBSET/MTC-ANN-2.0.1/mid/"

# Define metadata path
#METADATA_PATH = "/home/sirivasv/Documents/MuseScore3/Scores/MTRCV100/MTC-ANN-2.0.1/metadata/"
METADATA_PATH = "/media/sirivasv/JASON/Saul/MCC/DATASETS/DATASUBSET/MTC-ANN-2.0.1/metadata/"

# Define Query ID
query_song_id = "NLB070996_01"

In [14]:
# Read table of tune family
tune_family_filename = "MTC-ANN-tune-family-labels.csv"
tune_family_df = pd.read_csv(os.path.join(METADATA_PATH, tune_family_filename), header=None)
tune_family_df.head()

# Traverse musicxml files and tune family
song_id_x_family = {}
for root, directories, files in os.walk(DATASET_PATH):
    for file in files:
        song_id = file.split(".")[0]
        if (song_id not in song_id_x_family):
            song_id_x_family[song_id] = (file, tune_family_df[tune_family_df[0] == song_id].iloc[0][1])

# Remove the incomplete anotated tunes from the dataframe
reduced_tune_family_df = tune_family_df[tune_family_df[0].isin(list(song_id_x_family.keys()))]

In [15]:
# Read Files 
song_m21_streams = {}

# We traverse the reduced table
for query_row in reduced_tune_family_df.iterrows():
    tune_family_query = query_row[1][1]
    song_id_A = query_row[1][0]
    
    song_stream_A = m21.converter.parseFile(os.path.join(DATASET_PATH, song_id_x_family[song_id_A][0]))
    midi_tracks_A = m21.midi.translate.streamToMidiFile(song_stream_A)
    melody_w_times_A = getMelodyDeltaTimes(midi_tracks_A.tracks[0].events)
    
    song_m21_streams[song_id_A] = {
        "song_stream": song_stream_A,
        "midi_tracks": midi_tracks_A,
        "melody_w_times": melody_w_times_A
    }

In [16]:
# We traverse the shuffled table for the TEST
similarities_for_sort = []
for test_row in reduced_tune_family_df.iterrows():
    tune_family_test = test_row[1][1]
    key_test = test_row[1][0]

    if (key_test == query_song_id):
        continue
    
    melody_w_times_query = getMelodyDeltaTimes(
        song_m21_streams[query_song_id]["midi_tracks"].tracks[0].events)
    melody_w_times_test = getMelodyDeltaTimes(
        song_m21_streams[key_test]["midi_tracks"].tracks[0].events)
    
    similarity_distance = get_MTRC_v100_from_melody_w_times(melody_w_times_query, melody_w_times_test)
    
    similarities_for_sort.append((key_test, similarity_distance))

In [17]:
similarities_for_sort

[('NLB072587_01', 0.40522642679900744),
 ('NLB072587_02', 0.40521401985111666),
 ('NLB072774_02', 0.43050768542615486),
 ('NLB073046_01', 0.43053796419098145),
 ('NLB073588_01', 0.43054956896551727),
 ('NLB073672_01', 0.3925570913461539),
 ('NLB073681_01', 0.3925270432692308),
 ('NLB073743_01', 0.46854752218934914),
 ('NLB073822_01', 0.4432314560439561),
 ('NLB074004_01', 0.4558920940170941),
 ('NLB074048_02', 0.48118365384615386),
 ('NLB074227_01', 0.41785176282051284),
 ('NLB075551_01', 0.40522642679900744),
 ('NLB076625_01', 0.443226304945055),
 ('NLB076632_01', 0.4812144230769231),
 ('NLB144072_01', 0.6545889423076924),
 ('NLB070801_01', 0.5025570913461539),
 ('NLB072154_01', 0.6080576923076924),
 ('NLB072912_01', 0.5978934294871795),
 ('NLB072920_01', 0.686571906354515),
 ('NLB072946_01', 0.6419150641025642),
 ('NLB073426_01', 0.6039022435897436),
 ('NLB073929_01', 0.6799532585470086),
 ('NLB074028_01', 0.6165760869565218),
 ('NLB111656_01', 0.667252024291498),
 ('NLB112210_01', 0

In [18]:
similarities_for_sort.sort(key=lambda x: x[1])

In [22]:
[ (x[0], x[1], song_id_x_family[x[0]][1]) for x in similarities_for_sort]

[('NLB145525_01', 0.038, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB072895_01', 0.038002403846153845, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB072862_01', 0.10132867132867134, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB072837_01', 0.14518146447250924, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB072813_01', 0.20546675764647054, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB074575_01', 0.2210482071120369, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB072559_01', 0.2532412790697675, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB075184_01', 0.253259168157424, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB072898_01', 0.30392998027613416, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB075063_01', 0.32923661642411645, 'Het_was_op_een_driekoningenavond_1'),
 ('NLB074246_02', 0.3456208323410336, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB073628_01', 0.3507581375864958, 'Daar_was_laatstmaal_een_ruiter_2'),
 ('NLB072898_02', 0.3671952771493213, 'Er_reed_er_eens_een_ruiter_1'),
 ('NLB070693_01', 0.3672062621536218, 'Het_vrouwtje_van_Stavoren_1'),
 ('N

In [20]:
# Families
song_id_x_family["NLB070996_01"]

('NLB070996_01.musicxml', 'Er_reed_er_eens_een_ruiter_1')

In [21]:
song_id_x_family["NLB145525_01"]

('NLB145525_01.musicxml', 'Er_reed_er_eens_een_ruiter_1')

In [26]:
## NES ##
# NES_DATASET_PATH = "/home/sirivasv/Documents/DATASETS/nesmdb_midi/"
NES_DATASET_PATH = "/media/sirivasv/JASON/Saul/MCC/DATASETS/DATASUBSET/nesmdb_midi/"

In [27]:
# Traverse musicxml files and tune family
nes_song_filenames = []
for root, directories, files in os.walk(NES_DATASET_PATH):
    for file in files:
        nes_song_filenames.append(file)
nes_song_filenames

['134_GanbareGoemon2_17_18DeathbyBoilingHell.mid',
 '000_10_YardFight_00_01GameStart.mid',
 '000_10_YardFight_01_02GameOver.mid',
 '001_1942_00_01Start.mid',
 '001_1942_01_02MainBGM.mid',
 '001_1942_02_03Restart.mid',
 '001_1942_03_04MainBGMRestart.mid',
 '001_1942_04_05StageClear.mid',
 '001_1942_05_06GameOver.mid',
 '002_1943_TheBattleofMidway_00_01Title.mid',
 '002_1943_TheBattleofMidway_01_02PowerUpYourP38.mid',
 '002_1943_TheBattleofMidway_02_03MissionStart.mid',
 '002_1943_TheBattleofMidway_03_04AirBattleA.mid',
 '002_1943_TheBattleofMidway_04_05AntishipBattleA.mid',
 '002_1943_TheBattleofMidway_05_06MissionCompletedI.mid',
 '002_1943_TheBattleofMidway_06_07FailedintheAttack.mid',
 '002_1943_TheBattleofMidway_07_08AirBattleC.mid',
 '002_1943_TheBattleofMidway_08_09AntishipBattleC.mid',
 '002_1943_TheBattleofMidway_09_10AirBattleD.mid',
 '002_1943_TheBattleofMidway_10_11Daihiryu.mid',
 '002_1943_TheBattleofMidway_11_12Ayako.mid',
 '002_1943_TheBattleofMidway_13_14Ending.mid',
 '00

In [38]:
MAX_LIM_NES_SONGS = 1000

In [31]:
# Read Files 
nes_song_m21_streams = {}
len_nes_song_filenames = len(nes_song_filenames)
nes_songs_with_error = []

# We traverse the reduced table
cnt = 1
for query_row in nes_song_filenames:
    song_id_A = query_row
    try:
        
        song_stream_A = m21.converter.parseFile(os.path.join(NES_DATASET_PATH, song_id_A))
        midi_tracks_A = m21.midi.translate.streamToMidiFile(song_stream_A)
        melody_w_times_A = getMelodyDeltaTimes(midi_tracks_A.tracks[0].events)
    
        nes_song_m21_streams[song_id_A] = {
            "song_stream": song_stream_A,
            "midi_tracks": midi_tracks_A,
            "melody_w_times": melody_w_times_A
        }
        
    except:
        print("[ERROR!]")
        nes_songs_with_error.append(song_id_A)
    finally:
        print("{0}/{1} - {2}".format(cnt, len_nes_song_filenames, song_id_A))
        cnt+=1
        if (cnt == MAX_LIM_NES_SONGS):
            break
    

1/5278 - 134_GanbareGoemon2_17_18DeathbyBoilingHell.mid
2/5278 - 000_10_YardFight_00_01GameStart.mid
3/5278 - 000_10_YardFight_01_02GameOver.mid
4/5278 - 001_1942_00_01Start.mid
5/5278 - 001_1942_01_02MainBGM.mid
6/5278 - 001_1942_02_03Restart.mid
7/5278 - 001_1942_03_04MainBGMRestart.mid
8/5278 - 001_1942_04_05StageClear.mid
9/5278 - 001_1942_05_06GameOver.mid
10/5278 - 002_1943_TheBattleofMidway_00_01Title.mid
11/5278 - 002_1943_TheBattleofMidway_01_02PowerUpYourP38.mid
12/5278 - 002_1943_TheBattleofMidway_02_03MissionStart.mid
13/5278 - 002_1943_TheBattleofMidway_03_04AirBattleA.mid
14/5278 - 002_1943_TheBattleofMidway_04_05AntishipBattleA.mid
15/5278 - 002_1943_TheBattleofMidway_05_06MissionCompletedI.mid
16/5278 - 002_1943_TheBattleofMidway_06_07FailedintheAttack.mid
17/5278 - 002_1943_TheBattleofMidway_07_08AirBattleC.mid
18/5278 - 002_1943_TheBattleofMidway_08_09AntishipBattleC.mid
19/5278 - 002_1943_TheBattleofMidway_09_10AirBattleD.mid
20/5278 - 002_1943_TheBattleofMidway_10_1

155/5278 - 145_Golgo13_TopSecretEpisode_17_18AndThereWereNoneAct13.mid
156/5278 - 145_Golgo13_TopSecretEpisode_18_19Brain.mid
157/5278 - 145_Golgo13_TopSecretEpisode_19_20ActionStage3LastBoss.mid
158/5278 - 145_Golgo13_TopSecretEpisode_20_21Ending.mid
159/5278 - 145_Golgo13_TopSecretEpisode_21_22ToBeContinued.mid
160/5278 - 145_Golgo13_TopSecretEpisode_22_23Unused1.mid
161/5278 - 146_GomokuNarabeRenju_00_01TitleScreen.mid
162/5278 - 146_GomokuNarabeRenju_01_02GameStart.mid
163/5278 - 146_GomokuNarabeRenju_02_03GameWon.mid
164/5278 - 146_GomokuNarabeRenju_03_04GameDraw.mid
165/5278 - 146_GomokuNarabeRenju_04_05GameLost.mid
166/5278 - 146_GomokuNarabeRenju_05_06Player1Won.mid
167/5278 - 146_GomokuNarabeRenju_06_07Player2Won.mid
168/5278 - 147_Gradius_00_01Coin.mid
169/5278 - 147_Gradius_01_02BeginningofHistory.mid
170/5278 - 147_Gradius_02_03Challenger1985.mid
171/5278 - 158_HikarinoSenshiPhoton_WakuseiZoldiasnoTatakai_11_12Area2Battle.mid
172/5278 - 158_HikarinoSenshiPhoton_WakuseiZoldi

298/5278 - 155_HeraclesnoEikouII_TitannoMetsubou_08_09IntoTheUnderground.mid
299/5278 - 285_Route16Turbo_01_02Easy.mid
300/5278 - 285_Route16Turbo_02_03Normal.mid
301/5278 - 285_Route16Turbo_03_04Difficult.mid
302/5278 - 285_Route16Turbo_04_05Clear1.mid
303/5278 - 285_Route16Turbo_05_06Clear2.mid
304/5278 - 285_Route16Turbo_06_07GameOver.mid
305/5278 - 286_Rygar_00_01StartingPointSunset.mid
306/5278 - 286_Rygar_01_02StartingPointSunsetJPVersion.mid
307/5278 - 286_Rygar_02_03GranMountains.mid
308/5278 - 286_Rygar_03_04GranMountainsJPVersion.mid
309/5278 - 286_Rygar_04_05IndoraGod.mid
310/5278 - 286_Rygar_05_06IndoraGodJPVersion.mid
311/5278 - 286_Rygar_06_07Garloz.mid
312/5278 - 286_Rygar_08_09Boss.mid
313/5278 - 286_Rygar_09_10MtPrimevil.mid
314/5278 - 286_Rygar_10_11SagilasCave.mid
315/5278 - 286_Rygar_11_12SagilasCaveJPVersion.mid
316/5278 - 286_Rygar_12_13DoragosPalace.mid
317/5278 - 286_Rygar_13_14DoragosPalaceJPVersion.mid
318/5278 - 286_Rygar_14_15Lapis.mid
319/5278 - 286_Rygar_1

464/5278 - 021_AtlantisnoNazo_06_07GameOver.mid
465/5278 - 022_BabelnoTou_00_01TitleScreen.mid
466/5278 - 022_BabelnoTou_01_02FloorStart.mid
467/5278 - 022_BabelnoTou_02_03MainBGM.mid
468/5278 - 022_BabelnoTou_03_04FloorClear.mid
469/5278 - 022_BabelnoTou_04_05SpecialRoomBGM.mid
470/5278 - 022_BabelnoTou_05_06Miss.mid
471/5278 - 022_BabelnoTou_06_07Ending.mid
472/5278 - 023_BalloonFight_00_01GameStart.mid
473/5278 - 023_BalloonFight_01_02MainBGM.mid
474/5278 - 023_BalloonFight_02_03DroppingEnemy.mid
475/5278 - 023_BalloonFight_03_04Eaten.mid
476/5278 - 023_BalloonFight_04_05Restart.mid
477/5278 - 023_BalloonFight_05_06Clear.mid
478/5278 - 023_BalloonFight_06_07BalloonTripBonusStage.mid
479/5278 - 023_BalloonFight_07_08Perfect.mid
480/5278 - 023_BalloonFight_08_09Miss.mid
481/5278 - 023_BalloonFight_09_10GameOver.mid
482/5278 - 023_BalloonFight_10_11GetBubble.mid
483/5278 - 023_BalloonFight_11_12Pause.mid
484/5278 - 024_Baseball_00_01TitleScreen.mid
485/5278 - 024_Baseball_02_03Change2.

618/5278 - 036_BombermanII_01_02Title.mid
619/5278 - 036_BombermanII_02_03SelectGameMode.mid
620/5278 - 036_BombermanII_03_04StoryDemo.mid
621/5278 - 036_BombermanII_04_05GameStart.mid
622/5278 - 036_BombermanII_05_06Area135.mid
623/5278 - 036_BombermanII_06_07Area24.mid
624/5278 - 036_BombermanII_07_08Area6.mid
625/5278 - 036_BombermanII_08_09BGMInvincible.mid
626/5278 - 036_BombermanII_09_10BonusStageVSCompetitionBGM.mid
627/5278 - 036_BombermanII_10_11StageClear.mid
628/5278 - 036_BombermanII_11_12Ending.mid
629/5278 - 036_BombermanII_12_13CreditRoll.mid
630/5278 - 036_BombermanII_13_14PlayerOut.mid
631/5278 - 036_BombermanII_14_15GameOver.mid
632/5278 - 036_BombermanII_15_16VSVictory.mid
633/5278 - 036_BombermanII_16_17VSDraw.mid
634/5278 - 036_BombermanII_17_18VSChampionship.mid
635/5278 - 037_BoobyKids_00_01TitleScreen.mid
636/5278 - 040_BurgerTime_00_01RoundStart.mid
637/5278 - 040_BurgerTime_01_02MainTheme.mid
638/5278 - 040_BurgerTime_02_03RoundClear.mid
639/5278 - 040_BurgerT

756/5278 - 047_CastlevaniaII_Simon_sQuest_07_08MessageofDarkness.mid
757/5278 - 047_CastlevaniaII_Simon_sQuest_08_09ARequiem.mid
758/5278 - 048_Chack_nPop_00_01MainBGM.mid
759/5278 - 048_Chack_nPop_01_02RoundClearNoBonus.mid
760/5278 - 048_Chack_nPop_02_03RoundClearSpecialBonus.mid
761/5278 - 048_Chack_nPop_03_04RoundClearPerfectBonus.mid
762/5278 - 048_Chack_nPop_04_05Intermission.mid
763/5278 - 049_Challenger_00_01StoptheExpress.mid
764/5278 - 049_Challenger_01_02Trap.mid
765/5278 - 049_Challenger_02_03SearchPrincess.mid
766/5278 - 049_Challenger_03_04GetKeyword.mid
767/5278 - 049_Challenger_04_05RescuePrincess.mid
768/5278 - 050_ChaosWorld_00_01Title.mid
769/5278 - 050_ChaosWorld_01_02TheAdventureBegins.mid
770/5278 - 050_ChaosWorld_02_03RoyalCastle.mid
771/5278 - 050_ChaosWorld_03_04PeacefulTown.mid
772/5278 - 050_ChaosWorld_04_05Field.mid
773/5278 - 050_ChaosWorld_05_06Battle.mid
774/5278 - 050_ChaosWorld_06_07Victory.mid
775/5278 - 050_ChaosWorld_07_08UndergroundExploration.mid
7

911/5278 - 060_CrisisForce_02_03Stage16.mid
912/5278 - 060_CrisisForce_03_04Boss.mid
913/5278 - 060_CrisisForce_04_05StageClear.mid
914/5278 - 060_CrisisForce_05_06Stage25.mid
915/5278 - 060_CrisisForce_06_07Stage3.mid
916/5278 - 060_CrisisForce_07_08Stage4.mid
917/5278 - 060_CrisisForce_08_09Stage7.mid
918/5278 - 060_CrisisForce_09_10FinalBoss.mid
919/5278 - 060_CrisisForce_11_12GameOver.mid
920/5278 - 061_DarkLord_00_01Opening.mid
921/5278 - 061_DarkLord_01_02Title.mid
922/5278 - 061_DarkLord_02_03CharacterCreation.mid
923/5278 - 061_DarkLord_03_04TownofEastall.mid
924/5278 - 061_DarkLord_04_05LittleNeoFight.mid
925/5278 - 061_DarkLord_05_06Battle.mid
926/5278 - 061_DarkLord_06_07Victory.mid
927/5278 - 061_DarkLord_07_08ScenarioItem.mid
928/5278 - 061_DarkLord_08_09ScenarioClear.mid
929/5278 - 061_DarkLord_09_10TownofJanitor.mid
930/5278 - 061_DarkLord_10_11LostSoldier.mid
931/5278 - 061_DarkLord_11_12LittleDefenseLine.mid
932/5278 - 061_DarkLord_13_14CursedOrb.mid
933/5278 - 061_Dar

In [46]:
# "322_SuperMarioBros__02_03SwimmingAround.mid"
# "322_SuperMarioBros__10_11SavedthePrincess.mid"
# "339_Tetris_00_01TitleScreen.mid"

In [68]:
# song_stream_A = m21.converter.parseFile(os.path.join(DATASET_PATH, song_id_x_family["NLB070996_01"][0]))
song_stream_A = m21.converter.parseFile(os.path.join(NES_DATASET_PATH, "310_StarWars_TheEmpireStrikesBack_13_14Love.mid"))
midi_tracks_A = m21.midi.translate.streamToMidiFile(song_stream_A)
melody_w_times_query = getMelodyDeltaTimes(midi_tracks_A.tracks[0].events)

In [69]:
cnt = 1
# We traverse the shuffled table for the TEST
nes_similarities_for_sort = []
for test_row in nes_song_m21_streams:
    key_test = test_row

    if (key_test == query_song_id):
        continue
        
    melody_w_times_test = nes_song_m21_streams[key_test]["melody_w_times"]
    
    similarity_distance = get_MTRC_v100_from_melody_w_times(melody_w_times_query, melody_w_times_test)
    
    nes_similarities_for_sort.append((key_test, similarity_distance))
    print("{0}/{1}".format(cnt, MAX_LIM_NES_SONGS))
    cnt += 1

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

927/1000
928/1000
929/1000
930/1000
931/1000
932/1000
933/1000
934/1000
935/1000
936/1000
937/1000
938/1000
939/1000
940/1000
941/1000
942/1000
943/1000
944/1000
945/1000
946/1000
947/1000
948/1000
949/1000
950/1000
951/1000
952/1000
953/1000
954/1000
955/1000
956/1000
957/1000
958/1000
959/1000
960/1000
961/1000
962/1000
963/1000
964/1000
965/1000
966/1000
967/1000
968/1000
969/1000
970/1000
971/1000
972/1000
973/1000
974/1000
975/1000
976/1000
977/1000
978/1000
979/1000
980/1000
981/1000
982/1000
983/1000
984/1000
985/1000
986/1000
987/1000
988/1000
989/1000
990/1000
991/1000
992/1000
993/1000
994/1000
995/1000
996/1000
997/1000
998/1000


In [70]:
nes_similarities_for_sort

[('134_GanbareGoemon2_17_18DeathbyBoilingHell.mid', 0.7648370442294097),
 ('000_10_YardFight_00_01GameStart.mid', 0.7862104611280488),
 ('000_10_YardFight_01_02GameOver.mid', 0.8251092366252905),
 ('001_1942_00_01Start.mid', 0.9312300655794123),
 ('001_1942_01_02MainBGM.mid', 0.882404181184669),
 ('001_1942_02_03Restart.mid', 0.8443747580332946),
 ('001_1942_03_04MainBGMRestart.mid', 0.882404181184669),
 ('001_1942_04_05StageClear.mid', 0.8050824622531939),
 ('001_1942_05_06GameOver.mid', 0.747373693379791),
 ('002_1943_TheBattleofMidway_00_01Title.mid', 0.745881861725594),
 ('002_1943_TheBattleofMidway_01_02PowerUpYourP38.mid', 0.7672790181700307),
 ('002_1943_TheBattleofMidway_02_03MissionStart.mid', 0.7662911494628339),
 ('002_1943_TheBattleofMidway_03_04AirBattleA.mid', 0.8543259817017735),
 ('002_1943_TheBattleofMidway_04_05AntishipBattleA.mid', 0.771794630838048),
 ('002_1943_TheBattleofMidway_05_06MissionCompletedI.mid', 0.7784994192799073),
 ('002_1943_TheBattleofMidway_06_07Fa

In [71]:
nes_similarities_for_sort.sort(key=lambda x: x[1])

In [72]:
nes_similarities_for_sort

[('027_Batman_TheVideoGame_02_03Demo.mid', 0.43003631844683843),
 ('062_DeadlyTowers_12_13TheTower5.mid', 0.44391318133791463),
 ('039_BuraiFighter_08_09Miss.mid', 0.5295791853326697),
 ('158_HikarinoSenshiPhoton_WakuseiZoldiasnoTatakai_23_24LastBoss.mid',
  0.5403273755081301),
 ('035_Bomberman_08_09Miss.mid', 0.540744215134459),
 ('063_Deathbots_01_02Level12.mid', 0.5791672474943536),
 ('375_Vs_Excitebike_01_02StartingTrackSelection.mid', 0.5792809629394996),
 ('046_CastlevaniaIII_Dracula_sCurse_12_13Nightmare.mid', 0.5851377154289561),
 ('044_CastleofDragon_01_02OpeningTheDarklarzaCastle5Stage8.mid',
  0.5975461755701015),
 ('241_MiracleRopit_sAdventurein2100_05_06ThemeofUniverse.mid',
  0.6116641453877742),
 ('072_DonDokoDon_00_01TitleDemo.mid', 0.6120681378242354),
 ('062_DeadlyTowers_14_15TheTower7.mid', 0.614509856929624),
 ('092_DuckHunt_00_01TitleScreen.mid', 0.6179739321202735),
 ('055_CircusCaper_16_09bStage6.mid', 0.6218535554987805),
 ('071_DigitalDevilStory_MegamiTensei_0